In [1]:
import pandas as pd
import quandl
quandl.ApiConfig.api_key = '4nyydq5k_sPU89Nkvi_Q'
import matplotlib.pyplot as plt
import seaborn as sb

In [7]:
# Make calls to the quandl API to get the stock data for our five chosen stocks, for only the variables of interests, for only the first quarter, and put into stock df 
all_stock = quandl.get_table('WIKI/PRICES', ticker=["AAPL", "AMZN", "FB", "SBUX", "TWTR"], 
                             qopts={"columns":["date", "ticker","adj_open","adj_close","adj_volume"]},
                             date = {'gte': '2018-01-01', 'lte': '2018-03-31'}, paginate=True)

# Rename date column to make useful for future merges
all_stock = all_stock.rename(columns={'date':'Date'})
all_stock.head()

,Date,ticker,adj_open,adj_close,adj_volume
None,,,,,
0,2018-01-02,AAPL,170.16,172.26,25048048.0
1,2018-01-03,AAPL,172.53,172.23,28819653.0
2,2018-01-04,AAPL,172.54,173.03,22211345.0
3,2018-01-05,AAPL,173.44,175.00,23016177.0
4,2018-01-08,AAPL,174.35,174.35,20134092.0


In [3]:
# Make calls using the FinSents API wrapper (through quandl) for our 5 chosen companies on the NASDAQ and put into sentiments df
all_sent = quandl.get(['NS1/AAPL_US', 'NS1/AMZN_US', 'NS1/FB_US',  'NS1/SBUX_US', 'NS1/TWTR_US'], start_date='2018-01-01', end_date='2018-03-31')

# Reset index to make the date column usable for later merges
all_sent = all_sent.reset_index()
all_sent.head()

,Date,NS1/AAPL_US - Sentiment,NS1/AAPL_US - Sentiment High,NS1/AAPL_US - Sentiment Low,NS1/AAPL_US - News Volume,NS1/AAPL_US - News Buzz,NS1/AMZN_US - Sentiment,NS1/AMZN_US - Sentiment High,NS1/AMZN_US - Sentiment Low,NS1/AMZN_US - News Volume,...,NS1/SBUX_US - Sentiment,NS1/SBUX_US - Sentiment High,NS1/SBUX_US - Sentiment Low,NS1/SBUX_US - News Volume,NS1/SBUX_US - News Buzz,NS1/TWTR_US - Sentiment,NS1/TWTR_US - Sentiment High,NS1/TWTR_US - Sentiment Low,NS1/TWTR_US - News Volume,NS1/TWTR_US - News Buzz
0,2018-01-01,4.0,5.0,4.0,19.0,2.0,0.0,0.0,0.0,0.0,...,3.0,5.0,-2.0,6.0,5.0,4.0,5.0,3.0,156.0,10.0
1,2018-01-02,4.0,5.0,3.0,49.0,3.0,0.0,0.0,0.0,0.0,...,4.0,5.0,3.0,3.0,1.0,4.0,5.0,2.0,151.0,6.0
2,2018-01-03,4.0,5.0,4.0,41.0,3.0,0.0,0.0,0.0,0.0,...,5.0,5.0,4.0,4.0,1.0,4.0,5.0,2.0,156.0,6.0
3,2018-01-04,4.0,5.0,4.0,21.0,2.0,0.0,0.0,0.0,0.0,...,4.0,5.0,4.0,3.0,1.0,4.0,5.0,2.0,104.0,4.0
4,2018-01-05,4.0,5.0,4.0,64.0,9.0,0.0,0.0,0.0,0.0,...,4.0,5.0,4.0,14.0,10.0,4.0,5.0,2.0,149.0,6.0


In [4]:
# Drop unnecessary columns from sentiments df
cols = [2,3,7,8,12,13,17,18,22,23]
all_sent.drop(all_sent.columns[cols],axis=1,inplace=True)
all_sent.head()

,Date,NS1/AAPL_US - Sentiment,NS1/AAPL_US - News Volume,NS1/AAPL_US - News Buzz,NS1/AMZN_US - Sentiment,NS1/AMZN_US - News Volume,NS1/AMZN_US - News Buzz,NS1/FB_US - Sentiment,NS1/FB_US - News Volume,NS1/FB_US - News Buzz,NS1/SBUX_US - Sentiment,NS1/SBUX_US - News Volume,NS1/SBUX_US - News Buzz,NS1/TWTR_US - Sentiment,NS1/TWTR_US - News Volume,NS1/TWTR_US - News Buzz
0,2018-01-01,4.0,19.0,2.0,0.0,0.0,0.0,4.0,3.0,4.0,3.0,6.0,5.0,4.0,156.0,10.0
1,2018-01-02,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
2,2018-01-03,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
3,2018-01-04,4.0,21.0,2.0,0.0,0.0,0.0,4.0,10.0,3.0,4.0,3.0,1.0,4.0,104.0,4.0
4,2018-01-05,4.0,64.0,9.0,0.0,0.0,0.0,4.0,13.0,4.0,4.0,14.0,10.0,4.0,149.0,6.0


In [5]:
# Rename columns for readability 
all_sent_rename = all_sent.rename(index=str, columns={"NS1/AAPL_US - Sentiment": "AAPL Sentiment", 
                                    "NS1/AAPL_US - News Volume": "AAPL News Volume",
                                    "NS1/AAPL_US - News Buzz": "AAPL News Buzz",
                                    "NS1/AMZN_US - Sentiment": "AMZN Sentiment", 
                                    "NS1/AMZN_US - News Volume": "AMZN News Volume",
                                    "NS1/AMZN_US - News Buzz": "AMZN News Buzz",
                                    "NS1/FB_US - Sentiment": "FB Sentiment", 
                                    "NS1/FB_US - News Volume": "FB News Volume",
                                    "NS1/FB_US - News Buzz": "FB News Buzz",
                                    "NS1/SBUX_US - Sentiment": "SBUX Sentiment", 
                                    "NS1/SBUX_US - News Volume": "SBUX News Volume",
                                    "NS1/SBUX_US - News Buzz": "SBUX News Buzz",
                                    "NS1/TWTR_US - Sentiment": "TWTR Sentiment", 
                                    "NS1/TWTR_US - News Volume": "TWTR News Volume",
                                    "NS1/TWTR_US - News Buzz": "TWTR News Buzz"})
all_sent_rename.head(10)

,Date,AAPL Sentiment,AAPL News Volume,AAPL News Buzz,AMZN Sentiment,AMZN News Volume,AMZN News Buzz,FB Sentiment,FB News Volume,FB News Buzz,SBUX Sentiment,SBUX News Volume,SBUX News Buzz,TWTR Sentiment,TWTR News Volume,TWTR News Buzz
0,2018-01-01,4.0,19.0,2.0,0.0,0.0,0.0,4.0,3.0,4.0,3.0,6.0,5.0,4.0,156.0,10.0
1,2018-01-02,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
2,2018-01-03,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
3,2018-01-04,4.0,21.0,2.0,0.0,0.0,0.0,4.0,10.0,3.0,4.0,3.0,1.0,4.0,104.0,4.0
4,2018-01-05,4.0,64.0,9.0,0.0,0.0,0.0,4.0,13.0,4.0,4.0,14.0,10.0,4.0,149.0,6.0
5,2018-01-06,4.0,43.0,10.0,0.0,0.0,0.0,4.0,16.0,8.0,4.0,7.0,10.0,4.0,118.0,10.0
6,2018-01-07,5.0,9.0,10.0,0.0,0.0,0.0,4.0,9.0,10.0,4.0,3.0,10.0,4.0,55.0,10.0
7,2018-01-08,4.0,74.0,10.0,0.0,0.0,0.0,4.0,29.0,10.0,4.0,13.0,10.0,4.0,89.0,10.0
8,2018-01-09,4.0,52.0,8.0,0.0,0.0,0.0,4.0,43.0,10.0,4.0,10.0,8.0,4.0,69.0,5.0
9,2018-01-10,4.0,31.0,5.0,0.0,0.0,0.0,4.0,14.0,3.0,4.0,6.0,5.0,4.0,61.0,4.0


In [11]:
# Merge the stock data with the sentiments data 
merge_df = all_stock.merge(all_sent_rename, on="Date", how="outer")
merge_df.head(100)

,Date,ticker,adj_open,adj_close,adj_volume,AAPL Sentiment,AAPL News Volume,AAPL News Buzz,AMZN Sentiment,AMZN News Volume,AMZN News Buzz,FB Sentiment,FB News Volume,FB News Buzz,SBUX Sentiment,SBUX News Volume,SBUX News Buzz,TWTR Sentiment,TWTR News Volume,TWTR News Buzz
0,2018-01-02,AAPL,170.160,172.26,25048048.0,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
1,2018-01-02,AMZN,1172.000,1189.01,2587899.0,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
2,2018-01-02,FB,177.680,181.42,17694891.0,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
3,2018-01-02,SBUX,57.950,57.63,7199959.0,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
4,2018-01-02,TWTR,24.070,24.51,15283036.0,4.0,49.0,3.0,0.0,0.0,0.0,4.0,24.0,7.0,4.0,3.0,1.0,4.0,151.0,6.0
5,2018-01-03,AAPL,172.530,172.23,28819653.0,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
6,2018-01-03,AMZN,1188.300,1204.20,3044664.0,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
7,2018-01-03,FB,181.880,184.67,16595495.0,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
8,2018-01-03,SBUX,57.930,58.71,7417389.0,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
9,2018-01-03,TWTR,24.560,24.45,12792471.0,4.0,41.0,3.0,0.0,0.0,0.0,4.0,30.0,9.0,5.0,4.0,1.0,4.0,156.0,6.0
